In [ ]:
class TwitterScraper:
    tags = []
    tweet_data = []
    tweet_ids = set()
    
    def __init__(self, tags):
        self.tags = tags
        
    def create_webdriver_instance(self):
        options = EdgeOptions()
        options.use_chromium = True
        driver = Edge(options=options)
        return driver
    
    def get_tweet_data(card):
        username = card.find_element_by_xpath('.//span').text
        handle = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
        try:
            postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
        except NoSuchElementException:
            return
        content = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
        reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
        retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
        like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

        tweet = (username, handle, postdate, content, reply_cnt, retweet_cnt, like_cnt)
        return tweet
    
    def login_to_twitter(self):
        driver = self.create_webdriver_instance()
        #Navegar na tela de login
        driver.get('https://twitter.com/i/flow/login')

        #Espera implícita 
        wait = WebDriverWait(driver, 60)
        wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@name="text"]')))

        #Passar nome de usuário
        username = driver.find_element_by_xpath('//input[@name="text"]')
        username.send_keys('SousaC20')
        username.send_keys(Keys.RETURN)

        wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@name="password"]')))

        #Passar senha
        my_password = getpass()
        password = driver.find_element_by_xpath('//input[@name="password"]')
        password.send_keys(my_password)
        password.send_keys(Keys.RETURN)
        
        return driver

        
    def search_one_tag(self, tag):
        driver = self.login_to_twitter()
        
        wait = WebDriverWait(driver, 60)
        wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@aria-label="Consulta de busca"]')))

        #Pesquisar
        search_input = driver.find_element_by_xpath('//input[@aria-label="Consulta de busca"]')
        search_input.send_keys(tag)
        search_input.send_keys(Keys.RETURN)

        wait.until(EC.visibility_of_element_located((By.LINK_TEXT, 'Mais recentes')))
        
        #Pegar os tweets mais recents
        driver.find_element_by_link_text('Mais recentes').click()

        #Interar sobre cards e extrair os dados
        wait.until(EC.visibility_of_element_located((By.XPATH, '//article[@data-testid="tweet"]')))
        last_position = driver.execute_script('return window.pageYOffset')
        scrolling = True

        while scrolling and len(self.tweet_data) < 200:
            #Pegar tweets
            cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
            for card in cards[-15:]:
                try:
                    data = self.get_tweet_data(card)
                    if data:
                        tweet_id = ''.join(data)
                        if tweet_id not in self.tweet_ids:
                            self.tweet_ids.add(tweet_id)
                            self.tweet_data.append(data)
                except:
                    pass

            scroll_attemp = 0
            while True:
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
                sleep(1)

                curr_position = driver.execute_script('return window.pageYOffset')
                if curr_position == last_position:
                    scroll_attemp += 1

                    if scroll_attemp >= 3:
                        scrolling = False
                        break
                    else:
                        sleep(2)
                else:
                    last_position = curr_position
                    break
                    
    def search_tags(self):
        for tag in self.tags:
            self.search_one_tag(tag)

    def create_csv(self):
        with open('twitter_scraping_result.csv', 'w', newline = '', encoding = 'utf-8') as f:
            header = ['Username', 'Handle', 'Timestamp', 'Text', 'Replys', 'Retweets', 'Likes']
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerows(self.tweet_data)
            
            
scraper = TwitterScraper(['Rouanet'])
scraper.search_tags()
scraper.create_csv()
